In [ ]:
import ML4PS as ml
import numpy as np
from matplotlib import pyplot as plt

# Loading and preprocessing the data

In [ ]:
data_dir = '../../data/case14'

normalizer = ml.Normalizer(data_dir = data_dir, backend_name = 'pandapower')

interface = ml.Interface(data_dir = data_dir,
    backend_name = 'pandapower', batch_size = 1)

# Defining a simple Fully Connected Neural Network

In the following, we define a simple fully connected neural network.
It has been designed to input and output dictionnaries that respect the data formalism previously introduced.
As it cannot exploit the graph structure of the data, it does not make use of the adresses contained in $x$.

The option *hidden_dimensions* defines the dimension of the hidden variables.

In this case, we ask the neural network to output a prediction for bus voltage magnitude, knowing the active and reactive power of loads, and target power and target voltage of generators.

In [ ]:
input_features = {}
input_features['bus']       = ['in_service', 'max_vm_pu', 'min_vm_pu', 'vn_kv']
input_features['load']      = ['const_i_percent', 'const_z_percent', 'controllable', 'in_service', 
                               'p_mw', 'q_mvar', 'scaling', 'sn_mva']
input_features['gen']       = ['in_service', 'p_mw', 'scaling', 'sn_mva', 'vm_pu', 'slack', 'max_p_mw', 
                               'min_p_mw', 'max_q_mvar', 'min_q_mvar', 'slack_weight']
input_features['shunt']     = ['q_mvar', 'p_mw', 'vn_kv', 'step', 'max_step', 'in_service']
input_features['ext_grid']  = ['in_service', 'va_degree', 'vm_pu', 'max_p_mw', 'min_p_mw', 'max_q_mvar',
                               'min_q_mvar', 'slack_weight']
input_features['line']      = ['c_nf_per_km', 'df', 'g_us_per_km', 'in_service', 'length_km', 'max_i_ka',
                               'max_loading_percent', 'parallel', 'r_ohm_per_km', 'x_ohm_per_km']
input_features['trafo']     = ['df', 'i0_percent', 'in_service', 'max_loading_percent', 'parallel', 
                               'pfe_kw', 'shift_degree', 'sn_mva', 'tap_max', 'tap_neutral', 'tap_min', 
                               'tap_phase_shifter', 'tap_pos', 'tap_side', 'tap_step_degree', 
                               'tap_step_percent', 'vn_hv_kv', 'vn_lv_kv', 'vk_percent', 'vkr_percent']
input_features['poly_cost'] = ['cp0_eur', 'cp1_eur_per_mw', 'cp2_eur_per_mw2', 'cq0_eur', 
                               'cq1_eur_per_mvar', 'cq2_eur_per_mvar2']

output_features = {}
output_features['bus'] = ['res_vm_pu']

Since fully connected neural network can only work on vector of a given size, we have to first check the amount of objects of each class in the dataset, and to pass this piece of information to the neural network during init.

In [ ]:
a, x, nets = next(iter(interface.train))
n_obj = {k: np.max([np.shape(x_k_f)[1] for f, x_k_f in x[k].items()]) 
         for k in list(input_features.keys())+list(output_features.keys())}

In [ ]:
fully_connected = ml.Dense(input_features=input_features,
                        output_features=output_features,
                        n_obj=n_obj,
                        hidden_dimensions=[64])
fully_connected.save('my_fully_connected.pkl')

We may be interested in performing some complex postprocessing over the data. One can thus define a small PostProcessor class to transform the output of a neural network into an actually meaningful output. This will be useful when we start working with discrete variables for instance.

In [ ]:
class PostProcessor:
    def __call__(self, y):
        return {'bus': {'res_vm_pu': self.line_p1(y['bus']['res_vm_pu'])}}
    def line_p1(self, y):
        return 1.+1e-3*y
postprocessor = PostProcessor()

The following defines the loss function, which penalizes the discrepancy between the prediction of the neural network and the actual solution found using a power flow solver.

In [ ]:
def loss(params, x, y):
    y_hat = fully_connected.batch_forward(params, x)
    y_post = postprocessor(y_hat)
    loss_mag = ml.mean((y_post['bus']['res_vm_pu'] - y['bus']['res_vm_pu'])**2)
    return loss_mag

@ml.jit
def update(params, x, y, opt_state):
    value, grads = ml.value_and_grad(loss)(params, x, y)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [ ]:
step_size = 1e-2
opt_init, opt_update, get_params = ml.optimizers.adam(step_size)
opt_state = opt_init(fully_connected.weights)

The following defines the training loop that will iteratively pass through the train set and update the neural network weights accordingly.

In [ ]:
for epoch in range(5):
    
    # Training loop
    train_losses = []
    for adr, x, nets in interface.get_train_batch():
        x_norm = normalizer(x)
        interface.run_load_flow_batch(nets)
        y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
        fully_connected.weights, opt_state, train_loss = update(fully_connected.weights, 
            x_norm, y_truth, opt_state)
        train_losses.append(train_loss)
        #train_pbar.set_description("Epoch {}, Train loss = {:.2e}".format(epoch, train_loss))
        
    # Compute metrics over validation set
    val_losses = []
    for a, x, nets in interface.get_val_batch():
        x_norm = normalizer(x)
        interface.run_load_flow_batch(nets)
        y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
        val_loss = loss(fully_connected.weights, x_norm, y_truth)
        val_losses.append(val_loss)
    print("Epoch {}".format(epoch))
    print("    Train mean loss = {:.2e}".format(np.mean(train_losses)))
    print("    Validation mean loss = {:.2e}".format(np.mean(val_losses)))

# Compute metrics over the Test set

In [ ]:
losses = []
maes = []

for a, x, nets in interface.get_test_batch():
    
    # Perform prediction
    x_norm = normalizer(x)
    y_hat = fully_connected.batch_forward(fully_connected.weights, x_norm)
    y_post = postprocessor(y_hat)

    # Get ground truth
    interface.run_load_flow_batch(nets)
    y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
    
    # Compute metrics
    loss = np.mean((y_post['bus']['res_vm_pu'] - y_truth['bus']['res_vm_pu'])**2, axis=[1,2])
    mae = np.mean(np.mean(np.abs(y_post['bus']['res_vm_pu'] - y_truth['bus']['res_vm_pu']), axis=2), axis=1)
    losses.extend(list(loss))
    maes.extend(list(mae))
    
print('Loss')
print('    max        = {:.2e}'.format(np.max(losses)))
print('    90th perc. = {:.2e}'.format(np.percentile(losses, 90)))
print('    50th perc. = {:.2e}'.format(np.percentile(losses, 50)))
print('    10th perc. = {:.2e}'.format(np.percentile(losses, 10)))
print('    min        = {:.2e}'.format(np.min(losses)))
print('')
print('MAE')
print('    max        = {:.2e}'.format(np.max(maes)))
print('    90th perc. = {:.2e}'.format(np.percentile(maes, 90)))
print('    50th perc. = {:.2e}'.format(np.percentile(maes, 50)))
print('    10th perc. = {:.2e}'.format(np.percentile(maes, 10)))
print('    min        = {:.2e}'.format(np.min(maes)))

# Plot prediction against ground truth

In [ ]:
a, x, nets = next(iter(interface.test))

# Perform prediction
x_norm = normalizer(x)
y_hat = fully_connected.batch_forward(fully_connected.weights, x_norm)
y_post = postprocessor(y_hat)
y_post = np.reshape(y_post['bus']['res_vm_pu'], [-1])

# Get ground truth
interface.run_load_flow_batch(nets)
y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
y_truth = np.reshape(y_truth['bus']['res_vm_pu'], [-1])

# Compare results
plt.scatter(y_truth, y_post)
plt.xlabel('Ground truth')
plt.ylabel('Prediction')
plt.show()